In [1]:
from keras.optimizers import SGD
from keras.layers import Input, ZeroPadding2D
from keras.layers.core import Dense, Dropout, Activation
from keras.layers.convolutional import Convolution2D
from keras.layers.pooling import AveragePooling2D, GlobalAveragePooling2D, MaxPooling2D
from keras.layers import BatchNormalization
from tensorflow.keras.preprocessing import image
from keras.models import Model
import keras.backend as K 
from sklearn.metrics import log_loss
from custom_layers.scale_layer import Scale
import numpy as np
import os
import keras
import random
import cv2
import math
import seaborn as sns
from keras.preprocessing.image import ImageDataGenerator
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split   
from keras.layers import Concatenate
from tensorflow.keras.preprocessing.image import img_to_array

In [2]:
def densenet121_model(img_rows, img_cols, color_type=1, nb_dense_block=4, growth_rate=32, nb_filter=64, reduction=0.5, dropout_rate=0.0, weight_decay=1e-4, num_classes=None):
    '''
    DenseNet 121 Model for Keras

    Model Schema is based on 
    https://github.com/flyyufelix/DenseNet-Keras

    # Returns
        A Keras model instance.
    '''

    # Handle Dimension Ordering for different backends
    global concat_axis
    print("img dims")
    print(img_rows,img_cols)
    img_input = Input(shape=(img_rows, img_cols, color_type), name='data')
    concat_axis = 3


    # From architecture for ImageNet (Table 1 in the paper)
    nb_filter = 64
    nb_layers = [6,12,24,16] # For DenseNet-121

    # Initial convolution
    x = Convolution2D(filters=nb_filter, kernel_size=(7, 7), strides=(2, 2), padding='valid', 
           use_bias=False, name='conv1')(img_input)
    x = BatchNormalization(axis=concat_axis)(x)
    x = Scale(axis=concat_axis)(x)
    x = Activation('relu')(x)

    x = MaxPooling2D((3, 3), strides=(2, 2))(x)

    # Add dense blocks
    for block_idx in range(nb_dense_block - 1):
        stage = block_idx+2
        x, nb_filter = dense_block(x, stage, nb_layers[block_idx], nb_filter, growth_rate, dropout_rate=dropout_rate)

        # Add transition_block
        x = transition_block(x, stage, nb_filter, dropout_rate=dropout_rate)
        nb_filter = int(nb_filter)

    final_stage = stage + 1
    x, nb_filter = dense_block(x, final_stage, nb_layers[-1], nb_filter, growth_rate, dropout_rate=dropout_rate)

    x = BatchNormalization(axis=concat_axis)(x)
    x = Scale(axis=concat_axis )(x)
    x = Activation('relu')(x)

    x_fc = GlobalAveragePooling2D()(x)
    x_fc = Dense(1000)(x_fc)
    x_fc = Activation('softmax')(x_fc)

    model = Model(img_input, x_fc)

    # The method below works since pre-trained weights are stored in layers but not in the model
    x_newfc = GlobalAveragePooling2D()(x)
    x_newfc = Dense(num_classes)(x_newfc)
    x_newfc = Activation('softmax')(x_newfc)

    model = Model(img_input, x_newfc)

    # Learning rate is changed to 0.001
    sgd = SGD(lr=1e-3, decay=1e-6, momentum=0.9, nesterov=True)
    model.compile(optimizer=sgd, loss='categorical_crossentropy', metrics=['accuracy'])

    return model

In [3]:
def conv_block(x, stage, branch, nb_filter, dropout_rate=None):
    '''Apply BatchNorm, Relu, bottleneck 1x1 Conv2D, 3x3 Conv2D, and option dropout '''


    # 1x1 Convolution (Bottleneck layer)
    inter_channel = nb_filter * 4  
    x = BatchNormalization(axis=concat_axis)(x)
    x = Scale(axis=concat_axis)(x)
    x = Activation('relu')(x)
    x = Convolution2D(inter_channel, 1, 1)(x)

    if dropout_rate:
        x = Dropout(dropout_rate)(x)

    # 3x3 Convolution
    x = BatchNormalization(axis=concat_axis)(x)
    x = Scale(axis=concat_axis)(x)
    x = Activation('relu')(x)
    x = ZeroPadding2D((1, 1))(x)
    x = Convolution2D(nb_filter, 3, 3)(x)

    if dropout_rate:
        x = Dropout(dropout_rate)(x)

    return x


def transition_block(x, stage, nb_filter, dropout_rate=None):
    ''' Apply BatchNorm, 1x1 Convolution, averagePooling, optional compression, dropout  '''


    x = BatchNormalization(axis=concat_axis)(x)
    x = Scale(axis=concat_axis)(x)
    x = Activation('relu')(x)
    x = Convolution2D(int(nb_filter), 1, 1)(x)

    if dropout_rate:
        x = Dropout(dropout_rate)(x)

    x = AveragePooling2D((2, 2), strides=(2, 2))(x)

    return x


def dense_block(x, stage, nb_layers, nb_filter, growth_rate, dropout_rate=None, grow_nb_filters=True):
    ''' Build a dense_block where the output of each conv_block is fed to subsequent ones
        # Arguments
            x: input tensor
            stage: index for dense block
            nb_layers: the number of layers of conv_block to append to the model.
            nb_filter: number of filters
            growth_rate: growth rate
            grow_nb_filters: flag to decide to allow number of filters to grow
    '''

    concat_feat = x
    print("debug_line2")
    print(concat_feat)
    print(x)

    for i in range(nb_layers):
        branch = i + 1
        concat_feat = Concatenate(axis=concat_axis)([concat_feat, x])
        if grow_nb_filters:
            nb_filter += growth_rate
    x = conv_block(concat_feat, stage, branch, growth_rate, dropout_rate)

    return concat_feat, nb_filter

In [55]:
import os
import numpy as np
from tensorflow.keras.preprocessing import image
from tensorflow.keras.utils import to_categorical

def load_images_and_labels(base_dirs, target_size=(128, 128)):
    X_train = []
    Y_train = []
    X_test = []
    Y_test = []
    label_dict = {base_dir: idx for idx, base_dir in enumerate(base_dirs)}
    for base_dir in base_dirs:
        if os.path.isdir(base_dir):
            for file_name in sorted(os.listdir(base_dir)):
                file_path = os.path.join(base_dir, file_name)
                if file_name.lower().endswith(('.png', '.jpg', '.jpeg')):
                    img = image.load_img(file_path, target_size=target_size)
                    img_array = image.img_to_array(img)
                    if "Test\Test" not in base_dir:  # Train set
                        print(base_dir)
                        X_train.append(img_array)
                        Y_train.append(label_dict[base_dir])
                    else:  # Test set
                        X_test.append(img_array)
                        Y_test.append(label_dict[base_dir])
        else:
            print(f"Not a directory: {base_dir}")

    if not X_train:
        print("No training images found after processing.")
    else:
        print(f"Loaded {len(X_train)} training images successfully.")

    if not X_test:
        print("No test images found after processing.")
    else:
        print(f"Loaded {len(X_test)} test images successfully.")

    X_train = np.array(X_train)
    Y_train = to_categorical(np.array(Y_train), num_classes=len(base_dirs))
    X_test = np.array(X_test)
    Y_test = to_categorical(np.array(Y_test), num_classes=len(base_dirs))

    return X_train, Y_train, X_test, Y_test

# Example usage:
folder_paths = [
    # Training folders
    #r"Train\Train\Case", 
    #r"Train\Train\Ho_Entrance", 
    #r"Train\Train\Ho_Museum", 
    #r"Train\Train\McGregory_Lounge" 
    r"Train\Train\Lab", 
    r"Train\Train\Truck",
    # Test folder 
    #r"Test\Test\Case", 
    #r"Test\Test\Ho_Entrance", 
    #r"Test\Test\Ho_Museum", 
    r"Test\Test\Lab", 
    r"Test\Test\Truck"
]

X_train, Y_train, X_test, Y_test = load_images_and_labels(folder_paths)


Train\Train\Lab
Train\Train\Lab
Train\Train\Lab
Train\Train\Lab
Train\Train\Lab
Train\Train\Lab
Train\Train\Lab
Train\Train\Lab
Train\Train\Lab
Train\Train\Lab
Train\Train\Lab
Train\Train\Lab
Train\Train\Lab
Train\Train\Lab
Train\Train\Lab
Train\Train\Lab
Train\Train\Truck
Train\Train\Truck
Train\Train\Truck
Train\Train\Truck
Train\Train\Truck
Train\Train\Truck
Train\Train\Truck
Train\Train\Truck
Train\Train\Truck
Train\Train\Truck
Train\Train\Truck
Train\Train\Truck
Train\Train\Truck
Train\Train\Truck
Train\Train\Truck
Train\Train\Truck
Train\Train\Truck
Loaded 33 training images successfully.
Loaded 20 test images successfully.


In [56]:
from keras.callbacks import ModelCheckpoint
if __name__ == '__main__':
    
    img_rows, img_cols = 128,128
    channel = 3
    num_classes =1
    batch_size = 16 
    nb_epoch = 10


    # Load our model
    model = densenet121_model(img_rows=img_rows, img_cols=img_cols, color_type=channel, num_classes=num_classes)
    filepath="bestmodel.hdf5"
    checkpoint = ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=True, mode='max')
    callbacks_list = [checkpoint]
    # Start Fine-tuning
    history=model.fit(X_train, Y_train,
              batch_size=batch_size,
              epochs=nb_epoch,
              shuffle=True,
              verbose=1,
              validation_data=(X_valid, Y_valid),
              callbacks=callbacks_list,
              )

    # Make predictions
    predictions_valid = model.predict(X_valid, batch_size=batch_size, verbose=1)

    # Cross-entropy loss score
    score = log_loss(Y_valid, predictions_valid)

img dims
128 128
debug_line2
KerasTensor(type_spec=TensorSpec(shape=(None, 30, 30, 64), dtype=tf.float32, name=None), name='max_pooling2d_1/MaxPool:0', description="created by layer 'max_pooling2d_1'")
KerasTensor(type_spec=TensorSpec(shape=(None, 30, 30, 64), dtype=tf.float32, name=None), name='max_pooling2d_1/MaxPool:0', description="created by layer 'max_pooling2d_1'")
debug_line2
KerasTensor(type_spec=TensorSpec(shape=(None, 15, 15, 256), dtype=tf.float32, name=None), name='average_pooling2d_3/AvgPool:0', description="created by layer 'average_pooling2d_3'")
KerasTensor(type_spec=TensorSpec(shape=(None, 15, 15, 256), dtype=tf.float32, name=None), name='average_pooling2d_3/AvgPool:0', description="created by layer 'average_pooling2d_3'")
debug_line2
KerasTensor(type_spec=TensorSpec(shape=(None, 7, 7, 640), dtype=tf.float32, name=None), name='average_pooling2d_4/AvgPool:0', description="created by layer 'average_pooling2d_4'")
KerasTensor(type_spec=TensorSpec(shape=(None, 7, 7, 640),

NameError: name 'X_valid' is not defined